Imports

In [20]:
import gurobipy as gb
import pandas as pd
import numpy as np
import ast


Column Generation from Pre-processed Data

In [21]:
def indices_to_list(indices, length):
    return [1 if i in indices else 0 for i in range(length)]

In [22]:
data = pd.read_csv('data/combinations.tsv',sep='\t',index_col=False)
data['a'] = data['Members'].apply(ast.literal_eval).apply(lambda x: indices_to_list(x,30)) + data['Chairs'].apply(ast.literal_eval).apply(lambda x: indices_to_list(x,10))
data = data[['Day','Time','a']]
data['h'] = data.groupby(['Day', 'Time'])['a'].cumcount() + 1
data.info()

In [ ]:
data
#data[data['Day'] == 1]

,Day,Time,a,h
0,1,1,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1,1,1,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
2,1,1,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3
3,1,1,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4
4,1,1,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5
...,...,...,...,...
71581862,5,34,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",657796
71581863,5,34,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",657797
71581864,5,34,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",657798
71581865,5,34,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",657799


In [4]:
data.to_pickle('data/data.pkl')

The meat and potatoes of it all...

In [9]:
def maximize_meetings(data,filename='sol'):
    model = gb.Model("Meeting Scheduling")

    # Decision variables
    meetings = {}

    # Indices
    times = range(1,35)
    days = range(1,6)
    persons = range(0,40)

    #Generated columns hidden with dict{ day : dict{ time : [col_1,col_2,...col_h]}}
    cols = {
        1 : {}, #here would contain times 1-34, followed by a list of list/vectors/cols
        2 : {},
        3 : {},
        4 : {},
        5 : {}
    }
    
    print('building decision variables...')
    for d in days: # days 1-5
        for t in times: # time slots 1-34
            cols[d][t] = data[(data['Day'] == d) & (data['Time'] == t)]['a'].to_numpy()#.to_list()
            for h in range(len(cols[d][t])): #iterate over possible columns
                # Decision variables
                meetings[(t, d, h)] = model.addVar(vtype=gb.GRB.BINARY, name=f"x_{t}_{d}_{h}")

    print('building constraints ...')
    for d in days: # days 1-5
        for t in times: # time slots 1-34
            num_cols = len(cols[d][t])

            # Constraint 1: At most 4 meetings per time slot and day
            model.addConstr(gb.quicksum(meetings[(t, d, h)] for h in range(num_cols)) <= 4)

            # Constraint 2: Each participant attends at most one meeting per time slot and day
            for p in persons:
                model.addConstr(gb.quicksum(meetings[(t, d, h)] for h in range(num_cols)
                                      if cols[d][t][h][p] == 1) <= 1)
                
            # Constraint 3: No back-to-back meetings for participants
            if t != 34:
                for h in range(len(cols[d][t])):   
                    for h2 in range(len(cols[d][t+1])): # this is crucial as h varies!!!
                        for p in persons:
                            #print(t, d, h, p)
                            if cols[d][t][h][p] == 1 and cols[d][t+1][h2][p] == 1:
                                model.addConstr(meetings[(t, d, h)] + meetings[(t+1, d, h2)] <= 1)


    # Objective: Maximize number of meetings
    model.setObjective(gb.quicksum(meetings[(t, d, h)] for t in times
                                    for d in days for h in range(len(cols[d][t]))), gb.GRB.MAXIMIZE)

    print('solving model...')
    
    # Solve the optimization problem
    model.optimize()

    # Print solution
    if model.status == gb.GRB.OPTIMAL:
        with open(f'data/{filename}.tsv', 'w') as sol:
            sol.truncate(0)
            sol.write('D\tT\ta*\n')
            print("Optimal solution found!")
            for d in days:
                for t in times:
                    for h in range(len(cols[d][t])):
                        if meetings[(t, d, h)].x > 0.5:
                            sol.write(f"{d}\t{t}\t{cols[d][t][h]}\n")
    else:
        print("No solution found.")
    return model


In [42]:
m = maximize_meetings(data[data['h'] <= 500],'sol_lt_500')

solving model...
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 21.3.0 21D62)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50427552 rows, 77809 columns and 101232100 nonzeros
Model fingerprint: 0x66d5867b
Variable types: 0 continuous, 77809 integer (77809 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 89.0000000
Presolve removed 0 rows and 0 columns (presolve time = 6s) ...
Presolve removed 1987 rows and 0 columns (presolve time = 13s) ...
Presolve removed 1987 rows and 0 columns (presolve time = 15s) ...
Presolve removed 1987 rows and 0 columns (presolve time = 22s) ...
Presolve removed 1987 rows and 0 columns (presolve time = 25s) ...
Presolve removed 1987 rows and 0 columns (presolve time = 31s) ...
Presolve removed 1987 ro

### BABY DATA SOL:

In [16]:
baby_data2 = pd.read_csv('data/combinations2.tsv',sep='\t')
baby_data2['Chairs'] = baby_data2['Chairs'].apply(ast.literal_eval)
baby_data2['Members'] = baby_data2['Members'].apply(ast.literal_eval)
baby_data2['Chairs'] = baby_data2['Chairs'].apply(lambda x: indices_to_list(x,10))
baby_data2['Members'] = baby_data2['Members'].apply(lambda x: indices_to_list(x,30))
baby_data2['a'] = baby_data2['Members'] + baby_data2['Chairs']
baby_data2

,Day,Time,Members,Chairs,a
0,1,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1,1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1,1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,2,1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,2,2,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,2,3,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [17]:
baby_m2 = maximize_meetings(baby_data2,'baby_so2') 

building decision variables...
building constraints ...
solving model...
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 21.3.0 21D62)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6974 rows, 7 columns and 29 nonzeros
Model fingerprint: 0x1601ce41
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 4.0000000
Presolve removed 6970 rows and 3 columns
Presolve time: 0.01s
Presolved: 4 rows, 4 columns, 8 nonzeros
Variable types: 0 continuous, 4 integer (4 binary)

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+00, best bound 4

In [18]:
baby_data1 = pd.read_csv('data/combinations1.tsv',sep='\t')
baby_data1['Chairs'] = baby_data1['Chairs'].apply(ast.literal_eval)
baby_data1['Members'] = baby_data1['Members'].apply(ast.literal_eval)
baby_data1['Chairs'] = baby_data1['Chairs'].apply(lambda x: indices_to_list(x,10))
baby_data1['Members'] = baby_data1['Members'].apply(lambda x: indices_to_list(x,30))
baby_data1['a'] = baby_data1['Members'] + baby_data1['Chairs']
baby_data1

,Day,Time,Members,Chairs,a
0,1,1,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,4,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
baby_m1 = maximize_meetings(baby_data1,'baby_sol1') 

building decision variables...
building constraints ...
solving model...
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 21.3.0 21D62)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6970 rows, 2 columns and 8 nonzeros
Model fingerprint: 0x10c86697
Variable types: 0 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%
Optimal solution found!


### SINGLE DAY SOL:

In [ ]:
def maximize_meetings_on_day(data,filename='sol',day=0):
    model = gb.Model("Meeting Scheduling")

    # Decision variables
    meetings = {}

    # Indices
    times = range(1,35)
    persons = range(0,40)

    #Generated columns hidden with dict{ day : dict{ time : [col_1,col_2,...col_h]}}
    cols = {} #here would contain times 1-34, followed by a list of list/vectors/cols

    print('building decision variables ...')
    for t in times: # time slots 1-34
        cols[t] = data[(data['Time'] == t)]['a'].to_numpy()#.to_list()
        for h in range(len(cols[t])): #iterate over possible columns
            # Decision variables
            meetings[(t, h)] = model.addVar(vtype=gb.GRB.BINARY, name=f"x_{t}_{h}")

    print('building constraints ...')
    for t in times: # time slots 1-34
        num_cols = len(cols[t])

        print(f'building constraints (1) for t={t}...')
        # Constraint 1: At most 4 meetings per time slot and day
        model.addConstr(gb.quicksum(meetings[(t, h)] for h in range(num_cols)) <= 4)

        print(f'building constraints (2) for t={t}...')
        # Constraint 2: Each participant attends at most one meeting per time slot and day
        for p in persons:
            model.addConstr(gb.quicksum(meetings[(t, h)] for h in range(num_cols)
                                    if cols[t][h][p] == 1) <= 1)
            
        print(f'building constraints (3) for t={t}...')
        # Constraint 3: No back-to-back meetings for participants
        if t != 34:
            for h in range(len(cols[t])):   
                for h2 in range(len(cols[t+1])): # this is crucial as h varies!!!
                    for p in persons:
                        #print(t, h, p)
                        if cols[t][h][p] == 1 and cols[t+1][h2][p] == 1:
                            model.addConstr(meetings[(t, h)] + meetings[(t+1, h2)] <= 1)


    # Objective: Maximize number of meetings
    model.setObjective(gb.quicksum(meetings[(t, h)] for t in times
                                    for h in range(len(cols[t]))), gb.GRB.MAXIMIZE)

    print('solving model...')
    
    # Solve the optimization problem
    model.optimize()

    # Print solution
    if model.status == gb.GRB.OPTIMAL:
        with open(f'{filename}.tsv', 'w') as sol:
            sol.truncate(0)
            sol.write('D\tT\ta*\n')
            print("Optimal solution found!")
            for t in times:
                for h in range(len(cols[t])):
                    if meetings[(t, h)].x > 0.5:
                        sol.write(f"{day}\t{t}\t{cols[t][h]}\n")
    else:
        print("No solution found.")
    return model


In [17]:
m_d1 = maximize_meetings_on_day(data[data['Day'] == 1],'sol_day1',1)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-27


: 

#### LT 1200 Sols:

In [ ]:
data[(data['h'] <= 500) and (data['Day'] == 1)]

In [ ]:
m_d1_lt_1000 = maximize_meetings(data[(data['h'] <= 1500) and (data['Day'] == 1)],'sol_lt_1500_day_1')

In [ ]:
m_d1_lt_1000 = maximize_meetings(data[(data['h'] <= 1500) and (data['Day'] == 2)],'sol_lt_1500_day_2')

In [ ]:
m_d1_lt_1000 = maximize_meetings(data[(data['h'] <= 1500) and (data['Day'] == 3)],'sol_lt_1500_day_3')

In [ ]:
m_d1_lt_1000 = maximize_meetings(data[(data['h'] <= 1500) and (data['Day'] == 4)],'sol_lt_1500_day_4')

In [ ]:
m_d1_lt_1000 = maximize_meetings(data[(data['h'] <= 1500) and (data['Day'] == 5)],'sol_lt_1500_day_5')